In [1]:
import apache_beam as beam

In [2]:
covid_hist_path = ("data/HIST_PAINEL_COVIDBR_28set2020.csv")
estados_ibge_path = ("data/EstadosIBGE.csv")

In [3]:
class ToStr(beam.DoFn):
    '''
    Transforma uma tupla de listas em string
    
    Args:
        element (tupple): tupla entre coduf e lista de regiao, governador, uf, estado, casosAcumulado, obitosAcumulado
    Returns:
        strings
    '''
    def process(self, element):
        lista = element[1][0] + element[1][1]
        str1 = ','.join(map(str,lista))
        return [str1]

In [4]:
class GenerateCovidKey(beam.DoFn):
    '''
    Gera uma tupla de listas a partir dos elementos selecionados
    
    Args:
        element (list): lista =>  [coduf, estado, casosAcumulado, obitosAcumulado]
    
    Returns: 
        tupple => (coduf, estado, casosAcumulado, obitosAcumulado )
    
    '''
    def process(self, element):
        return [(element[0], [element[1], element[2], element[3], element[4]])]

In [5]:
class GenerateEstadosKey(beam.DoFn):
    '''
    Gera uma tupla de listas a partir dos elementos selecionados
    
    Args:
        element (list): lista =>  [coduf, governador, uf]
    
    Returns: 
        tupple => (coduf, governador, uf)
    
    '''
    
    def process(self, element):
        return [(element[0], [element[1], element[2]])]

In [6]:
class SelectEstadosFields(beam.DoFn):
    '''
    Seleciona dentre todas as features extraídas, código, governador, uf
    
    Args:
        element(list): lista => [...] todas as features
    
    Returns:
        lista => [coduf, governador, uf]
    
    '''
    def process(self, element):
        return [element[1] + ',' + element[0] + ',' + element[3]]

In [7]:
class SelectCovidFields(beam.DoFn):
    '''
    Seleciona dentre todas as features extraídas, coduf, regiao, estado, casosAcumulado, obitosAcumulado
    
    Args:
        element(list): lista => [...] todas as features
    
    Returns:
        lista => [coduf, regiao, estado, casosAcumulado, obitosAcumulado]
    
    '''
    def process(self, element):
        return [element[3] + ',' + element[0] + ',' + element[1] + ',' + element[10] + ',' + element[12]]

In [8]:
class SplitRow(beam.DoFn):
    '''
    Transforma dados csv em listas
    
    Args:
        element(str): todas as features
    
    Returns:
        lista => [...] todas as features
    
    '''
    def process(self, element):
        new_element = element.split(';')
        yield new_element

In [9]:
pipe = beam.Pipeline()

In [10]:
covid_data = (pipe
     |"Extrair csv" >> beam.io.ReadFromText(covid_hist_path, skip_header_lines=True)
     |"Transformar dados csv em listas" >> beam.ParDo(SplitRow())
     |"Filtrar dados nacionais" >> beam.Filter(lambda x:x[3]!='76')
     |"Filtrar dados municipais" >> beam.Filter(lambda x:x[2]=='')
     |"Filtrar dados municipais resultantes" >> beam.Filter(lambda x:x[4]=='')
     |"Extrair apennas a ultima amostragem" >> beam.Filter(lambda x:x[7]=="28/09/2020")
     |"Selecionar apenas features relevantes" >> beam.ParDo(SelectCovidFields())
     |"Retornar listas formatadas" >> beam.Map(lambda x:x.split(","))
     |"Gera uma lista de tuplas" >> beam.ParDo(GenerateCovidKey())
)

In [11]:
estados_data = (pipe
    |"Extrair csv" >> beam.io.ReadFromText(estados_ibge_path, skip_header_lines=True)
    |"Transformar dados csv em listas" >> beam.ParDo(SplitRow())
    |"Selecionar apenas features relevantes" >> beam.ParDo(SelectEstadosFields())
    |"Retornar listas formatadas" >> beam.Map(lambda x:x.split(","))
    |"Gera uma lista de tuplas" >> beam.ParDo(GenerateEstadosKey())
)

In [12]:
merged_data = ((covid_data,estados_data)
    |"Agrupa as duas Pcollections em uma nova Pcollection" >> beam.Flatten()
    |"Agrupa os dados por código de estados" >> beam.GroupByKey()
    |"Transforma uma tupla de listas em string" >> beam.ParDo(ToStr())
    |"Envia os dados para result_file.csv" >> beam.io.WriteToText("data/result_file",".csv")
)

In [ ]:
pipe.run()